<a href="https://colab.research.google.com/github/drscook/m5364_23sp_data_science1/blob/main/pipelines_AirBNB_ipynb_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Free Cloud Computing 
## It's not just π in the sky
## April 19, 2023
## Tarleton State University
## Educational Excellence Week
### Dr. Scott Cook
### Chief Data Scientist & Assoc Prof of Mathematics

In [ ]:
# upgrade to most recent versions (Pandas 2.0 released on April 6, 2023)
# run once at start of session
# ignore any "pip dependency" errors
! pip install -q --upgrade numpy pandas scikit-learn matplotlib shap jupyter-autotime
# restart kernel so upgrade takes effect
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

Major Scientific Python Packages
1. Numpy - https://numpy.org/ - Foundational scientific computing package (arrays)
1. Pandas - https://pandas.pydata.org/ - Most important data science package (dataframes)
1. Scikit-Learn - https://scikit-learn.org - Machine Learning
1. Jupyter Notebook - https://jupyter.org/ - Front end user interface you are using now
1. Google Colab - https://colab.research.google.com/ - Cloud compute platform you are using now





Let's do a little machine learning project using the Boston Airbnb dataset from OpenML.org https://www.openml.org/search?type=data&status=active&id=43819

In [ ]:
from sklearn.datasets import fetch_openml
airbnb = fetch_openml(data_id=43819, parser="auto")
airbnb.data.head(3)

We could directly use as is. But, most projects use data stored in a local file, like a csv. So, to demonstrate how to read a csv, we will write to a csv and then read it back in.

Warning: Colab borrows a machine from Google for you to use.  But Google wants it back when you are done. This Jupyter notebook automatically saves to your Google Drive (folder "Colab Notebook"). But any other file you upload or create will be deleted unless you mount and save to your google drive.

There are 2 ways to mount Drive.
1. Try cell below
1. If it errs, click the folder icon on far left under {x}, then click icon of folder with triangle (says Mount Drive when you hover over it)

In [ ]:
import pathlib, google.colab, pandas as pd
root = pathlib.Path("/content/drive")
google.colab.drive.mount(str(root))

Cell below 
1. creates a folder named "CEE_workshop" in drive/MyDrive
1. saves data to a csv file
1. reads data back in

In [ ]:
path = root / "MyDrive/CEE_workshop"
path.mkdir(exist_ok=True)
file = path / "airbnb.csv"
airbnb.data.to_csv(file, index=False)  # saves to your drive
df = pd.read_csv(file) # reads that file back in

If you need to read a csv, simply upload it to google drive, edit the path to point at it, and use read_csv. Pandas supports other file formats like excel, parquet, json, pickle, hdf, and more.

We typically spend a lot of time doing data exploration and visualization at the start of a project so we can make smart choices about data wrangling and modeling approaches. Python has some fantastic [visualization packages](https://pyviz.org/overviews/index.html) like [Bokeh](https://docs.bokeh.org/), [Seaborn](https://seaborn.pydata.org/), [Plotly](https://plotly.com/python/), [Matplotlib](https://matplotlib.org/), and many others.

There are many great tutorials online for that. So, for the sake of time, we'll assume you've completed this step and skip to wrangling and modeling.

Again, the goal of this workshop is to show what is possible using Colab.  It is not meant to teach Python. So don't worry about understanding all the code below.

Our task: Predict "review_scores_value".  This is the overall rating for an airbnb property and combines the other "review_scores_X" columns.  So, we can't use those columns as inputs (features) ... that would be cheating.

In [ ]:
import pathlib, numpy as np, pandas as pd, sklearn, missingno as msno
from sklearn import set_config
set_config(transform_output = "pandas")

df = pd.read_csv(file).convert_dtypes().set_index('id') # convert_dtypes activates new "nullable" datatypes in pandas https://pandas.pydata.org/pandas-docs/version/1.1/user_guide/integer_na.html

# little helper to easily show the first 3 lines as we progress through data wrangling
def disp(X):
    display(X.head(3))

# force all strings to lower case
# Python style guides prefer "easier to ask forgivenes than permission" over "look before you leap"
# Some columns are not strings. We simply attempt to lower case everything and fail gracefully for non-strings.
for col in df.columns:
    try:
        df[col] = df[col].str.lower()
    except AttributeError:
        pass

# price has entries like "6,000.00". The comma forces this to be a string rather than float.  Let's fix that
df['price'] = df['price'].str.replace(',','').astype(float)
disp(df)

house_rules is free-form text of the rules of the property. Let's try to extract something useful from it.

In [ ]:
for x in df['house_rules'][100:104]:
    print(x)
    print()

In [ ]:
# Let's try to deterimine if pets and smoking are allowed
# We guess that pets are allowed unless "no pets" appears in house_rules.
# This is clearly too crude and will miss other expressions like "pets are not allowed".
# We can rethink this later if the resulting model does not perform well.
df['pets_ok'] = ~df['house_rules'].str.contains('no pets')

# During data exploration, we observed anytime smoking was discussed in house_rules, it was prohibited.
# In other words, we did not see explicit permission to smoke.
# We will make a crude assumption that smoking is prohibited anytime "smok" appears in house_rules
# We can rethink this later if the resulting model does not perform well.
df['smoking_ok'] = ~df['house_rules'].str.contains('smok')

# If we wanted to be more careful, we could search for multiple expressions like this:
# df['smoking_ok'] = ~df['house_rules'].str.contains('|'.join(['smoking', 'non smoking', 'non-smoking', 'smoking not permitted', 'smoking is not permitted']))
# https://stackoverflow.com/questions/48541444/pandas-filtering-for-multiple-substrings-in-series/48600345#48600345

In [ ]:
# Look for missing data
msno.bar(df)

In [ ]:
# Correlations of missing data among pairs of columns
msno.heatmap(df)

In [ ]:
target = 'review_scores_rating'

# decide which columns we will use as features (inputs) in the model
features = [
    'price',
    'bedrooms',
    'bathrooms',
    'cleaning_fee',
    'pets_ok',
    'smoking_ok',
    'amenities_dict',
    'neighbourhood_cleansed',
]
numeric_features = ['price', 'bedrooms', 'bathrooms', 'cleaning_fee', 'pets_ok', 'smoking_ok']  # which features are already numeric

mask = df[target].notnull() # find columns where target is NOT missing
F = df.loc[mask, features]  # feature dataframe of feature columns and rows where target is not missing
y = df.loc[mask, target]    # target dataframe of target column and rows where target is not missing
msno.bar(F)

amenities_dict looks important - it indicates which amenities the property offers.  However, it has been stored in a rather awkward format.  Let's extract it as indicator (dummy) variables.

In [ ]:
for x in F['amenities_dict'][:3]:
    print(x)

In [ ]:
# extract amenities into indicator columns
A = F['amenities_dict'].str.split(', ', expand=True)  # split pieces of amentities string into separate columns
disp(A)

amenities_features = A.columns = [x.split(':')[0].strip("'") for x in A.iloc[0]]  # get columns names from row 0
A.columns = amenities_features
disp(A)

def amentity_offered(x):
    return (x.str[-1] == '1').astype('boolean')  # return True if the final character is 1
A = A.apply(amentity_offered)
disp(A)

Neighborhood might also be important. Most machine learners can't handle categorical variables like this directly, but we can "one-hot-encode" into indicator (dummy) variables.  https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)  # create one-hot encoder object
N = enc.fit_transform(F[['neighbourhood_cleansed']])#.astype('boolean')  # apply object to neighborhood_cleansed column
disp(N)

# we don't need the "neighborhood_cleansed_" prefix here, so let's remove it by keeping only charaters 23 and beyond
neighorhoods_features = N.columns.str[23:]
N.columns = neighorhoods_features
disp(N)

In [ ]:
# join and drop the original columns
X = F.join(N).join(A).drop(columns=['amenities_dict', 'neighbourhood_cleansed'])
disp(X)

display(X.describe(include='all'))

## We are done with data wrangling.  Let's compact this code into one cell for convenience.

In [ ]:
%reload_ext autotime
import pathlib, google.colab, numpy as np, pandas as pd, sklearn, shap, matplotlib.pyplot as plt, missingno as msno
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_openml
from sklearn import set_config
set_config(transform_output = "pandas")
def disp(X):
    display(X.head(3))

root = pathlib.Path("/content/drive")
google.colab.drive.mount(str(root))
path = root / "MyDrive/CEE_workshop"
path.mkdir(exist_ok=True)
file = path / "airbnb.csv"

try:
    df = pd.read_csv(file)
    print(f'using data saved at {file}')
except:
    print(f'donwloading data')
    airbnb = fetch_openml(data_id=43819, parser="auto")
    df = airbnb.data
    df.to_csv(file, index=False)
df = df.convert_dtypes().set_index('id')

for col in df.columns:
    try:
        df[col] = df[col].str.lower()
    except AttributeError:
        pass
df['price'] = df['price'].str.replace(',','').astype(float)
df['pets_ok'] = ~df['house_rules'].str.contains('no pets')
df['smoking_ok'] = ~df['house_rules'].str.contains('smok')

target = 'review_scores_rating'
features         = ['price', 'bedrooms', 'bathrooms', 'cleaning_fee', 'pets_ok', 'smoking_ok', 'amenities_dict', 'neighbourhood_cleansed']
numeric_features = ['price', 'bedrooms', 'bathrooms', 'cleaning_fee', 'pets_ok', 'smoking_ok']
mask = df[target].notnull()
F = df.loc[mask, features]

A = df['amenities_dict'].str.split(', ', expand=True)
amenities_features = A.columns = [x.split(':')[0].strip("'") for x in A.iloc[0]]
A.columns = amenities_features
def amentity_offered(x):
    return (x.str[-1] == '1').astype('boolean')
A = A.apply(amentity_offered)

enc = OneHotEncoder(sparse_output=False)
N = enc.fit_transform(df[['neighbourhood_cleansed']]).astype('boolean')
neighorhoods_features = N.columns.str[23:]
N.columns = neighorhoods_features

X = F.join(N).join(A).drop(columns=['amenities_dict', 'neighbourhood_cleansed'])
disp(X)

Now we can start building a model. I decided not to use neighborhoods after all because there are a lot of them (25) and I'm worried about the [Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality).

I have the same concern about amenities, so we'll use Principal Components Analysis to reduce their dimension. But I don't think that will work for neighborhoods, so we'll exclude neighborhood for now. Perhaps a future version will include them.

In [ ]:
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import joblib

# create target
y = df.loc[mask, target]

# set random seed for reproducible results
seed = 42

# create holdout set to estimate generalization performance after tuning
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.10, random_state=seed)

# create preprocessing pipeline
# Rescale numerics to [0,1]
numeric_prep = Pipeline(steps = [('scaler', MinMaxScaler())])

# Principal Components Analysis to reduce dimensionality of 133 amenities 
amenities_prep = Pipeline(steps = [('pca', PCA())])

# combine ouput of numeric & amenities pipelines
combined_prep = ColumnTransformer(transformers = [('numeric', numeric_prep, numeric_features), ('amenities', amenities_prep, amenities_features)])

# Use K-nearest neighbors imputer to fill missing values
prep = Pipeline(steps = [('combined', combined_prep), ('imputer', KNNImputer())])
display(prep)

# dictionary to store models
models = dict()

# function to make training with different supervised learners (regressors) convenient
def run_model(learner, hyperparameters):
    # append final regressor to pipeline
    estimator = Pipeline(steps = [('prep', prep), ('learner', learner)])
    
    # prepare to tune hyperparameters using either exhaustive grid search or randomized grid search
    model = RandomizedSearchCV(estimator, hyperparameters, cv=5, scoring='r2')
    
    # give model a name based on its regessor
    model.name = str(learner).split('(')[0]

    print(f'Fitting {model.name} with hyperparameter grid:', hyperparameters)
    model.fit(X_train, y_train)

    # Apply best model to holdout set to estimate generalization performance
    model.generalization_score = model.score(X_holdout, y_holdout)
    print(f'best model: R^2={model.generalization_score:.3f} with hyperparameters {model.best_params_}')

    y_pred = model.predict(X_holdout)
    p = [np.min([y_pred, y_holdout]), np.max([y_pred, y_holdout])]
    plt.plot(p, p, 'black')
    plt.plot(y_holdout, y_pred, '.')

    # save file to google drive folder
    file = path / f'{model.name}.model'
    model.file = file
    joblib.dump(model, file)
    return model

This is our complete preprocessing pipeline. We just need one final step - the supervised learning algorithm. Since the target variable is continuous, we are doing a *regression* task.

There are many supervised learning algorithms in Scikit-Learn, so we may want to try several of them. Let's write a helper function to make this easy.

Many of these steps involve tunable hyperparameters where there is not a clear "best" option. We want to try many different settings for these hyperparameters to see what performs best. This would take signifcant coding to implement yourself. Luckily, Scikit-Learn offer "GridSearchCV" and "RandomizedSearchCV" which automate this process. We just give it a dictionary of hyperparameters with values to try and it will loop over them to find the best.

In [ ]:
# K-nearest neighbors
from sklearn.neighbors import KNeighborsRegressor
learner = KNeighborsRegressor()
hyperparameters = {
    'prep__combined__amenities__pca__n_components': [2,4],
    'prep__imputer__n_neighbors': [3,7],
    'learner__n_neighbors': [3,6,9],
}
model = run_model(learner, hyperparameters)
models[model.name] = model

In [ ]:
# Support Vector Machines
from sklearn.svm import SVR
learner = SVR()
hyperparameters = {
    'prep__combined__amenities__pca__n_components': [2,4],
    'prep__imputer__n_neighbors': [3,7],
    'learner__C': np.linspace(0.1,5,3),
}
model = run_model(learner, hyperparameters)
models[model.name] = model

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
learner = RandomForestRegressor(random_state=seed)
hyperparameters = {
    'prep__combined__amenities__pca__n_components': [2,4],
    'prep__imputer__n_neighbors': [3,7],
    'learner__max_depth': [3,6,9],
}
model = run_model(learner, hyperparameters)
models[model.name] = model

Well, these models suck. R^2 ranges from 0 (terrible) to 1 (perfect). Even our best model is around 0.2.  So, we want to go back and rethink some choices made along the way. But we built this code so that will probably not require much effort or recoding.

## Now, let's bin the target and demonstrate a classification task

In [ ]:
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, RocCurveDisplay
import joblib

y = df.loc[mask, target]
# In order to make this a classification task, we will use pandas qcut function to bin the target values into n_labels classes with roughly equal frequencies
n_labels = 2
y_binned = pd.qcut(df.loc[mask, target], q=n_labels, precision=1)
y    = y_binned.cat.codes
bins = y_binned.cat.categories.astype(str).to_list()

# set random seed for reproducible results
seed = 42

# create holdout set to estimate generalization performance after tuning
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.10, random_state=seed)

# create preprocessing pipeline
# Rescale numerics to [0,1]
numeric_prep = Pipeline(steps = [('scaler', MinMaxScaler())])

# Principal Components Analysis to reduce dimensionality of 133 amenities 
amenities_prep = Pipeline(steps = [('pca', PCA())])

# combine ouput of numeric & amenities pipelines
combined_prep = ColumnTransformer(transformers = [('numeric', numeric_prep, numeric_features), ('amenities', amenities_prep, amenities_features)])

# Use K-nearest neighbors imputer to fill missing values
prep = Pipeline(steps = [('combined', combined_prep), ('imputer', KNNImputer())])
display(prep)

# dictionary to store models
models = dict()

# function to make training with different supervised learners (regressors) convenient
def run_model(learner, hyperparameters):
    # append final regressor to pipeline
    estimator = Pipeline(steps = [('prep', prep), ('learner', learner)])
    
    # prepare to tune hyperparameters using either exhaustive grid search or randomized grid search
    model = GridSearchCV(estimator, hyperparameters, cv=5, scoring='f1_micro')
    
    # give model a name based on its regessor
    model.name = str(learner).split('(')[0]

    print(f'Fitting {model.name} with hyperparameter grid:', hyperparameters)
    model.fit(X_train, y_train)

    # Apply best model to holdout set to estimate generalization performance
    model.generalization_score = model.score(X_holdout, y_holdout)
    print(f'best model: F1={model.generalization_score:.3f} with hyperparameters {model.best_params_}')

    print(classification_report(y_holdout, model.predict(X_holdout), target_names=bins))

    ConfusionMatrixDisplay.from_estimator(model, X_holdout, y_holdout, display_labels=bins)
    plt.show()
    
    if n_labels == 2:
        RocCurveDisplay.from_estimator(model, X_holdout, y_holdout)
        plt.plot([0,1],[0,1])
        plt.show()

    # save file to google drive folder
    file = path / f'{model.name}.model'
    model.file = file
    joblib.dump(model, file)
    return model
    

In [ ]:
# K-nearest neighbors
from sklearn.neighbors import KNeighborsClassifier
learner = KNeighborsClassifier()
hyperparameters = {
    'prep__combined__amenities__pca__n_components': [2,4],
    'prep__imputer__n_neighbors': [3,7],
    'learner__n_neighbors': [3,6,9],
}
model = run_model(learner, hyperparameters)
models[model.name] = model

In [ ]:
# Support Vector Machines
from sklearn.svm import SVC
learner = SVC()
hyperparameters = {
    'prep__combined__amenities__pca__n_components': [2,4],
    'prep__imputer__n_neighbors': [3,7],
    'learner__C': np.linspace(0.1,5,3),
}
model = run_model(learner, hyperparameters)
models[model.name] = model

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
learner = RandomForestClassifier(random_state=seed)
hyperparameters = {
    'prep__combined__amenities__pca__n_components': [2,4],
    'prep__imputer__n_neighbors': [3,7],
    'learner__max_depth': [3,6,9],
}
model = run_model(learner, hyperparameters)
models[model.name] = model

- How students submit
    - Share & send link link (email or canvas)
    - Post to Github (not private by default)
    - Download & send .ipynb (email or canvas)
    - Create & send static html file: https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab


My favorite (free) books are:
- [WhirlwindTourOfPython](https://github.com/jakevdp/WhirlwindTourOfPython)
- [PythonDataScienceHandbook](https://github.com/jakevdp/PythonDataScienceHandbook)

The internet is full of great Python & data science materials. Sadly, they are all out of date (to some extent) before they are published. Code examples often don't work because a package it uses has changed. But that's the price we pay for an amazingly rich ecosystem of free and powerful tools.